In [1]:
import pandas as pd
import csv
import glob
from sklearn.compose import make_column_selector, ColumnTransformer

In [2]:
!pip install kaggle

In [2]:
# function to load the csv files that have been scraped from FBREF as df
def extract_FBREF(subdirectories):
    for i,sub in enumerate(subdirectories):
        if i==0:
            for index,csv in enumerate(glob.glob(f"{sub}/*.csv")):
                if index==0:
                    extracted_data=pd.read_csv(csv, index_col=0)
                else:
                    extracted_data= pd.concat([extracted_data, pd.read_csv(csv)], ignore_index=True)
        else: 
            for csv in glob.glob(f"{sub}/*.csv"):
                extracted_data= pd.concat([extracted_data, pd.read_csv(csv)], ignore_index=True)
    return extracted_data
    
# function to load data from kaggle dataset Transfermarkt as df
def extract_TM():
    
    if not os.path.exists("data_TM"):
        os.makedirs("data_TM")
    
    kaggle.api.authenticate()
    kaggle.api.dataset_download_files('davidcariboo/player-scores', path='data_TM', unzip=True)
    
    player_valuations=pd.read_csv("data_TM/player_valuations.csv")
    players=pd.read_csv("data_TM/players.csv")

    return player_valuations, players

In [3]:
# combine prior functions
def extract(subdirectories):
    player_stats=extract_FBREF(subdirectories)
    valuations,players=extract_TM()
    return player_stats, valuations, players

In [4]:
# function to transform player stats (scraped from FBREF)
def transform_stats(df):
    #format the df properly
    df.iloc[0] = df.iloc[0].fillna('Saison')
    header = df.iloc[0]
    df = df[1:]
    df = df.rename(columns=header)

    #delete unwanted columns
    df=df.drop("Spiele", axis=1)
    df = df.loc[:, ~df.columns.duplicated(keep="first")]

    #remove duplicate rows of header row
    index_dup_header = df[df["Geboren"] == "Geboren"].index
    df = df.drop(index_dup_header)

    #change datatype of numeric columns
    columns_to_convert = df.columns[5:27]

    for column in columns_to_convert:
        df[column] = df[column].astype(float)
        
    return df

In [5]:
#function to transform player stats from kaggle dataset Transfermarkt
def transform_TM(valuations, players):
    def rename_or_drop(date):
            if date.month >= 5 and date.month <= 7 and date.day >= 15:
                return f"{date.year-2001}_{date.year-2000}"
            else:
                return None 
    valuations['date']=pd.to_datetime(valuations['date'])
    valuations['Saison'] = valuations['date'].apply(rename_or_drop)
    valuations = valuations.dropna(subset=['Saison'])
    valuations = valuations.rename(columns={'player_id': 'Spieler_id', 'market_value_in_eur':'Marktwert', 'player_club_domestic_competition_id':'Liga'})
    valuations=valuations[['Spieler_id','Saison', 'Marktwert', 'Liga']]
    valuations = valuations[valuations['Liga'].isin(['GB1', 'L1', 'ES1', 'FR1', 'IT1'])]


    players['Spieler']=players['first_name']+" "+players['last_name']
    players['Spieler_id']=players['player_id']
    players=players[['Spieler', 'Spieler_id']]
    
    merged_df = pd.merge(players,valuations, on='Spieler_id', how='inner')
    merged_df = merged_df.drop('Spieler_id', axis=1)
    
    return merged_df

In [6]:
# function to combine both Transformers and inner join the results
def transform(df1, df2, df3):
    stats=transform_stats(df1)
    valuations=transform_TM(df2, df3)
    final= pd.merge(stats,valuations, on=['Spieler', 'Saison'], how='inner')

    return final

In [7]:
# function to load the end result as csv that is ready to be worked with by Data Sccientist (me)
def load(df, file_name):
    df.to_csv(file_name, index=False)

In [8]:
#ETL process
subs=["22_23","21_22","20_21","19_20","18_19","17_18"]
stats, valuations, players= extract(subs)
whole_table=transform(stats, valuations, players)
load(whole_table, "Spieler_Marktwerte.csv")